In [1]:
from dependency import *
from pathlib import Path
bundle = Bundle(Path("/home/algor/.p2/"), "org.eclipse.xtext")
bundle.update_jars()
bundle.jars
bundle.get_manifest_file_lines()
bundle.update_dependencies()
print(bundle)
print(bundle.get_exported_dependencies())

org.eclipse.xtext | 2 jars | 14 deps
[org.eclipse.emf.ecore.xmi : exp True, org.eclipse.emf.ecore : exp True, org.antlr.runtime : exp True, com.google.inject : exp True, org.eclipse.emf.mwe.core : exp True, org.eclipse.emf.mwe.utils : exp True, org.eclipse.xtext.util : exp True]


In [2]:
dependencies = parse_manifest_file_lines(test_deps)
bundle = dependencies[0].to_bundle(Path("/home/algor/.p2/"))
print(bundle)
bundle.update_jars()
print(bundle)
print(bundle.jars)
bundle.update_dependencies()
print(bundle.dependencies)
print(bundle)
print(f"exported deps {bundle.get_exported_dependencies()}")

org.eclipse.emf.ecore.xmi | 0 jars | 0 deps
org.eclipse.emf.ecore.xmi | 2 jars | 0 deps
['org.eclipse.emf.ecore.xmi_2.16.0.v20190528-0725.jar', 'org.eclipse.emf.ecore.xmi.source_2.16.0.v20190528-0725.jar']
[org.eclipse.core.runtime : exp False, org.eclipse.emf.ecore : exp True]
org.eclipse.emf.ecore.xmi | 2 jars | 2 deps
exported deps [org.eclipse.emf.ecore : exp True]


In [6]:
bundles = list(map(lambda x: Bundle(Path("/home/algor/git/raox"),x), ["ru.bmstu.rk9.rao", "ru.bmstu.rk9.rao.lib", "ru.bmstu.rk9.rao.ui"]))
for i in bundles:
    i.update_jars()
    i.update_dependencies()
    print(i)

AttributeError: 'NoneType' object has no attribute 'seek'

In [4]:
bundles

[ru.bmstu.rk9.rao | 0 jars | 0 deps,
 ru.bmstu.rk9.rao.lib | 0 jars | 0 deps,
 ru.bmstu.rk9.rao.ui | 0 jars | 0 deps]